In [34]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq litellm==1.72.6 --progress-bar off
!pip install -qqq pydantic==2.11.7 --progress-bar off
!pip install -qqq numpydoc==1.8.0 --progress-bar off

In [1]:
import json
import textwrap
from typing import Literal

import litellm
from dotenv import load_dotenv
from litellm import completion
from pydantic import BaseModel
from rich.pretty import pprint

load_dotenv()

PROMPT = "Define what is Deep Learning in one sentence"

messages = [{"content": PROMPT, "role": "user"}]

## Prompt Multiple Providers


In [3]:
%%time

response = completion(model="gpt-4.1-mini", messages=messages)

CPU times: user 8.26 ms, sys: 3.06 ms, total: 11.3 ms
Wall time: 1.22 s


In [4]:
pprint(response)

ModelResponse(
│   id='chatcmpl-BjfDZZWIvZhSp9YxRaab0uvgG82wV',
│   created=1750222901,
│   model='gpt-4.1-mini-2025-04-14',
│   object='chat.completion',
│   system_fingerprint='fp_6f2eabb9a5',
│   choices=[
│   │   Choices(
│   │   │   finish_reason='stop',
│   │   │   index=0,
│   │   │   message=Message(
│   │   │   │   content='Deep Learning is a subset of machine learning that uses multi-layered neural networks to automatically learn hierarchical representations from large amounts of data.',
│   │   │   │   role='assistant',
│   │   │   │   tool_calls=None,
│   │   │   │   function_call=None,
│   │   │   │   provider_specific_fields={'refusal': None},
│   │   │   │   annotations=[]
│   │   │   ),
│   │   │   provider_specific_fields={}
│   │   )
│   ],
│   usage=Usage(
│   │   completion_tokens=26,
│   │   prompt_tokens=15,
│   │   total_tokens=41,
│   │   completion_tokens_details=CompletionTokensDetailsWrapper(
│   │   │   accepted_prediction_tokens=0,
│   │   │   audio_tokens=0,
│   │   │   reasoning_tokens=0,
│   │   │   rejected_prediction_tokens=0,
│   │   │   text_tokens=None
│   │   ),
│   │   prompt_tokens_details=PromptTokensDetailsWrapper(
│   │   │   audio_tokens=0,
│   │   │   cached_tokens=0,
│   │   │   text_tokens=None,
│   │   │   image_tokens=None
│   │   )
│   ),
│   service_tier='default'
)

In [5]:
pprint(response.usage)

Usage(
│   completion_tokens=26,
│   prompt_tokens=15,
│   total_tokens=41,
│   completion_tokens_details=CompletionTokensDetailsWrapper(
│   │   accepted_prediction_tokens=0,
│   │   audio_tokens=0,
│   │   reasoning_tokens=0,
│   │   rejected_prediction_tokens=0,
│   │   text_tokens=None
│   ),
│   prompt_tokens_details=PromptTokensDetailsWrapper(
│   │   audio_tokens=0,
│   │   cached_tokens=0,
│   │   text_tokens=None,
│   │   image_tokens=None
│   )
)

In [6]:
print(textwrap.fill(response.choices[0].message.content, 120))

Deep Learning is a subset of machine learning that uses multi-layered neural networks to automatically learn
hierarchical representations from large amounts of data.


In [7]:
%%time

response = completion(
    model="gemini/gemini-2.5-flash", messages=messages, reasoning_effort="disable"
)

CPU times: user 35.4 ms, sys: 14.4 ms, total: 49.8 ms
Wall time: 825 ms


In [8]:
print(textwrap.fill(response.choices[0].message.content, 120))

Deep Learning is a subfield of machine learning that uses neural networks with multiple layers to learn complex patterns
from data.


## Structured Output


In [10]:
%%time

litellm.enable_json_schema_validation = True

PROMPT = """
Classify the text sentiment into one of negative, neutral or positive.
Give your reasoning in the `reasoning` field.

Text:
```
I am very happy to say that AI has taken my job, for good!
```
"""

messages = [{"role": "user", "content": PROMPT}]


class SentimentClassification(BaseModel):
    sentiment: Literal["negative", "neutral", "positive"]
    reasoning: str


response = completion(
    model="gemini/gemini-2.5-flash",
    messages=messages,
    response_format=SentimentClassification,
)

CPU times: user 76.9 ms, sys: 21.2 ms, total: 98.1 ms
Wall time: 2.27 s


In [11]:
print(response.choices[0].message.content)

{
  "sentiment": "positive",
  "reasoning": "The text explicitly states 'I am very happy to say that AI has taken my job, for good!'. The phrase 'very happy' clearly indicates a positive emotion, and 'for good!' implies a beneficial and permanent change, reinforcing the positive sentiment despite the seemingly negative event of losing a job."
}


In [12]:
json_content = response.choices[0].message.content
pprint(SentimentClassification.model_validate_json(json_content))

SentimentClassification(
│   sentiment='positive',
│   reasoning="The text explicitly states 'I am very happy to say that AI has taken my job, for good!'. The phrase 'very happy' clearly indicates a positive emotion, and 'for good!' implies a beneficial and permanent change, reinforcing the positive sentiment despite the seemingly negative event of losing a job."
)

## Tool Calling


In [13]:
def estimate_house_price(
    square_meters: int, n_bedrooms: int, expensive_location: bool
) -> int:
    """Get house price estimate in USD

    Parameters
    ----------
    square_meters : int
        Square meters of the house
    n_bedrooms : int
        Number of bedrooms
    expensive_location: bool
        Whether the house is located in an expensive location

    Returns
    -------
    int
        the estimated price of the house in USD
    """
    return (
        (square_meters * 1_000) + (n_bedrooms * 50_000) + (expensive_location * 250_000)
    )

In [15]:
%%time

tools = [litellm.utils.function_to_dict(estimate_house_price)]

PROMPT = """
I'm looking at a 3 bedroom house at San Jose.

It's about 250 square meters. How much should I get it for?
"""

messages = [{"role": "user", "content": PROMPT}]

response = completion(
    model="gemini/gemini-2.5-flash", messages=messages, tools=tools, tool_choice="auto"
)

CPU times: user 24.8 ms, sys: 10.9 ms, total: 35.7 ms
Wall time: 1.35 s


In [16]:
pprint(tools)

[
│   {
│   │   'name': 'estimate_house_price',
│   │   'description': 'Get house price estimate in USD',
│   │   'parameters': {
│   │   │   'type': 'object',
│   │   │   'properties': {
│   │   │   │   'square_meters': {'type': 'integer', 'description': 'Square meters of the house'},
│   │   │   │   'n_bedrooms': {'type': 'integer', 'description': 'Number of bedrooms'},
│   │   │   │   'expensive_location': {'type': 'boolean'}
│   │   │   },
│   │   │   'required': ['square_meters', 'n_bedrooms', 'expensive_location']
│   │   }
│   }
]

In [17]:
pprint(response.choices[0].message.tool_calls)

[
│   ChatCompletionMessageToolCall(
│   │   index=0,
│   │   function=Function(
│   │   │   arguments='{"expensive_location": true, "n_bedrooms": 3, "square_meters": 250}',
│   │   │   name='estimate_house_price'
│   │   ),
│   │   id='call_fed1051e-09cd-4d36-9e5c-83a140c03a2a',
│   │   type='function'
│   )
]

In [18]:
response_message = response.choices[0].message
messages.append(response_message.to_dict())

tool_calls = response_message.tool_calls
available_functions = {
    "estimate_house_price": estimate_house_price,
}

In [19]:
for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(**function_args)
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": str(function_response),
        }
    )

In [20]:
pprint(messages)

[
│   {
│   │   'role': 'user',
│   │   'content': "\nI'm looking at a 3 bedroom house at San Jose.\n\nIt's about 250 square meters. How much should I get it for?\n"
│   },
│   {
│   │   'content': None,
│   │   'role': 'assistant',
│   │   'tool_calls': [
│   │   │   {
│   │   │   │   'index': 0,
│   │   │   │   'function': {
│   │   │   │   │   'arguments': '{"expensive_location": true, "n_bedrooms": 3, "square_meters": 250}',
│   │   │   │   │   'name': 'estimate_house_price'
│   │   │   │   },
│   │   │   │   'id': 'call_fed1051e-09cd-4d36-9e5c-83a140c03a2a',
│   │   │   │   'type': 'function'
│   │   │   }
│   │   ],
│   │   'function_call': None
│   },
│   {
│   │   'tool_call_id': 'call_fed1051e-09cd-4d36-9e5c-83a140c03a2a',
│   │   'role': 'tool',
│   │   'name': 'estimate_house_price',
│   │   'content': '650000'
│   }
]

In [21]:
final_response = litellm.completion(
    model="gemini/gemini-2.5-flash",
    messages=messages,
)

In [22]:
print(final_response.choices[0].message.content)

For a 3-bedroom house of 250 square meters in San Jose, you should expect to pay around $650,000.
